In [1]:
import os

In [2]:
%pwd

'D:\\Python\\MLProjectsPW\\CreditCardFraud1\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'D:\\Python\\MLProjectsPW\\CreditCardFraud1'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name: str
    criterion: str
    max_features: str
    n_estimators: int

In [6]:
from CreditCardFraud.constants import *
from CreditCardFraud.utils.common import read_yaml , create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml (config_filepath)
        self.params = read_yaml (params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            criterion= params.criterion,
            max_features=params.max_features,
            n_estimators=params.n_estimators
        )
        
        return model_trainer_config

In [8]:
import pandas as pd
from sklearn.ensemble import  RandomForestClassifier
import joblib
from sklearn.metrics import  accuracy_score , classification_report
from CreditCardFraud.logging import logger

In [9]:
class ModelTrainer:
    def __init__(self, config=ModelTrainerConfig):
        self.config = config
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        target_column = train_data.columns[-1]

        train_x = train_data.drop([target_column],axis=1)
        test_x = test_data.drop([target_column],axis=1)
        train_y = train_data[target_column]
        test_y = test_data[target_column]

        model = RandomForestClassifier(criterion=self.config.criterion, max_features=self.config.max_features, n_estimators=self.config.n_estimators,random_state=42)
        model.fit(train_x,train_y)
        
        y_pred = model.predict(test_x)
        
        accuracy = accuracy_score(test_y, y_pred)
        cr = classification_report(test_y,y_pred)
        
        logger.info(f"""The accuracy of {model} is {accuracy} \n\n
        classification report is {cr}""")
        

        joblib.dump(model,os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-08-13 00:36:12,324: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-13 00:36:12,340: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-13 00:36:12,340: INFO: common: creating directory at: artifacts]
[2023-08-13 00:36:12,340: INFO: common: creating directory at: artifacts/model_trainer]
[2023-08-13 00:36:13,056: INFO: 1381227063: The accuracy of RandomForestClassifier(max_features='log2', n_estimators=200, random_state=42) is 0.8007968127490039 


        classification report is               precision    recall  f1-score   support

         0.0       0.82      0.96      0.88       197
         1.0       0.60      0.22      0.32        54

    accuracy                           0.80       251
   macro avg       0.71      0.59      0.60       251
weighted avg       0.77      0.80      0.76       251
]
